In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [63]:
import ipy_progressbar

In [87]:
import re
import pprint

In [81]:
STMSTOPS = "data/stops.txt"
BASICBIZ = "yelp_total.csv"

In [3]:
stops = pd.read_csv(STMSTOPS)

In [18]:
metro_re = re.compile(r'http://www\.stm\.info/metro/')
bus_re = re.compile(r'http://m\.stm\.info/fr/horaires/arrets/\d+/bus')

In [19]:
imetro = stops.stop_url.str.match(metro_re)

In [34]:
stop_type = []
for url in stops.stop_url:
    if metro_re.match(url):
        stop_type.append('metro')
    elif bus_re.match(url):
        stop_type.append('bus')
    else:
        stop_type.append(None)
stops['stop_type'] = stop_type
stops['stop_type'] = stops['stop_type'].astype('category')

In [56]:
stops.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,wheelchair_boarding,stop_type
0,1,10280,Station Henri-Bourassa,45.556109,-73.667452,http://www.stm.info/metro/M01.htm,1,metro
1,10,10262,Station Sherbrooke,45.518345,-73.568251,http://www.stm.info/metro/M10.htm,2,metro
2,100006,61396,Sainte-Anne / Sainte-Marie,45.402742,-73.947489,http://m.stm.info/fr/horaires/arrets/61396/bus,2,bus
3,100007,61503,Robert / Cluny,45.580406,-73.607774,http://m.stm.info/fr/horaires/arrets/61503/bus,1,bus
4,100009,61506,du Bois-de-Boulogne / stationnement,45.541494,-73.689973,http://m.stm.info/fr/horaires/arrets/61506/bus,2,bus


In [82]:
business = pd.read_csv(BASICBIZ, index_col='business_id')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (79,91,93,94,95,96,109) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [92]:
business.head()

,Unnamed: 0,full_address,hours.Friday.close,hours.Friday.open,hours.Tuesday.close,hours.Tuesday.open,hours.Thursday.close,hours.Thursday.open,hours.Wednesday.close,hours.Wednesday.open,...,checkin_info.2-4,checkin_info.2-1,checkin_info.2-5,checkin_info.1-1,checkin_info.23-6,checkin_info.23-3,checkin_info.0-3,checkin_info.1-2,checkin_info.1-4,checkin_info.0-0
business_id,,,,,,,,,,,,,,,,,,,,,
lW9Up8oVTWUEut_R6nZG-g,1,3606 Rue Ontario Est\nRosemont-La Petite-Patri...,00:00,08:00,00:00,08:00,00:00,08:00,00:00,08:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RZma06CqXbOIMzTQL-eJFQ,2,"10715 Boulevard Pie\nMontréal-Nord\nMontréal, ...",01:00,11:00,23:00,11:00,01:00,11:00,23:00,11:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GxfgoiDLfs7HC8C_wFsGPw,3,3001 Masson Rue\nRosemont-La Petite-Patrie\nMo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yLj-pKIc-V530j1G6ySQ2Q,4,5300 rue Sherbrooke E\nRosemont-La Petite-Patr...,23:00,11:30,21:00,11:30,21:00,11:30,21:00,11:30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IY5GFZTZCywC0ylKuG9ebA,5,2901 Sherbrooke Rue E\nRosemont-La Petite-Patr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
ismetro = stops['stop_type'] == 'metro'
metros = stops[ismetro]
isbus = stops['stop_type'] == 'bus'
buses = stops[isbus]

In [94]:
mindist = []
for lon, lat in ipy_progressbar.ProgressBar(zip(business.longitude, business.latitude)):
    result = []
    for stp in [metros, buses]:
        dlon = lon - stp.stop_lon
        dlat = lat - stp.stop_lat
        dists = np.sqrt(dlon**2 + dlat**2)
        imin = np.argmin(dists)
        result.extend((dists[imin], stp.stop_id.ix[imin], stp.stop_type.ix[imin]))
    mindist.append(result)

In [95]:
stm_prox = pd.DataFrame(mindist, columns=('closest_mstop_dist', 'closest_mstop_id', 'closest_mstop_type', 'closest_bstop_dist', 'closest_bstop_id', 'closest_bstop_type'), index=business.index)

In [96]:
stm_prox.head()

,closest_mstop_dist,closest_mstop_id,closest_mstop_type,closest_bstop_dist,closest_bstop_id,closest_bstop_type
business_id,,,,,,
lW9Up8oVTWUEut_R6nZG-g,0.007192,25,metro,0.000308,120961,bus
RZma06CqXbOIMzTQL-eJFQ,0.043264,3,metro,0.000325,1800551,bus
GxfgoiDLfs7HC8C_wFsGPw,0.021392,26,metro,0.000157,115294,bus
yLj-pKIc-V530j1G6ySQ2Q,0.006017,22,metro,0.000637,119900,bus
IY5GFZTZCywC0ylKuG9ebA,0.005044,26,metro,0.000849,118724,bus


In [97]:
stm_prox.to_csv("data/closest_bus-metro_stops.csv")

In [98]:
!head data/closest_bus-metro_stops.csv

business_id,closest_mstop_dist,closest_mstop_id,closest_mstop_type,closest_bstop_dist,closest_bstop_id,closest_bstop_type
lW9Up8oVTWUEut_R6nZG-g,0.00719181921543,25,metro,0.000308031699746,120961,bus
RZma06CqXbOIMzTQL-eJFQ,0.0432638167741,3,metro,0.000325377104303,1800551,bus
GxfgoiDLfs7HC8C_wFsGPw,0.0213923534101,26,metro,0.000156605300042,115294,bus
yLj-pKIc-V530j1G6ySQ2Q,0.00601657463346,22,metro,0.000636730963596,119900,bus
IY5GFZTZCywC0ylKuG9ebA,0.00504357888112,26,metro,0.000849245977327,118724,bus
RySnetMQUJKbbtliTt4Nmw,0.00280648348508,23,metro,0.00255223642557,142122,bus
IYdjcA92wjvM5RtPZ3C7kw,0.0230580935248,19,metro,0.00260055859384,1602252,bus
pvbBfLIVJ_5sJXUTAKmkng,0.0284838283417,21,metro,6.97036584413e-05,1701551,bus
A9yOopsy2bQgStF3Y6lSyA,0.0110805705201,23,metro,0.000245861037986,121572,bus
